# Capillary theory

I propose a theory to explain whether we observe dimple or not in a thin liquid film. The core idea is the dimple is unstable because surface tension always has a tendency to smooth it out. To observe it, the surface tension induced flow has to be sufficiently small, so that the dimple stays longer. 

The dimple formation around a beet slice has two steps:

1. Initial dimple formation due to the suction flow of the beet slice;
2. The smooth-out due to the surface tension induced flow. 

In this notebook, I will model these two steps based on the physical picture depicted above. A simple numerical solution will be presented.